In [1]:
import os
from datetime import datetime
import time
import numpy as np
import h5py
import random
from PIL import Image
import tensorflow as tf
import json
from model import Model

In [2]:
def build_batch(path_to_tfrecords_file, num_examples, batch_size, shuffled):
    assert tf.gfile.Exists(path_to_tfrecords_file), '%s not found' % path_to_tfrecords_file

    filename_queue = tf.train.string_input_producer([path_to_tfrecords_file], num_epochs=None)
    #image, length, digits = read_and_decode(filename_queue)

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
            serialized_example,
            features={
                'image': tf.FixedLenFeature([], tf.string),
                'length': tf.FixedLenFeature([], tf.int64),
                'digits': tf.FixedLenFeature([5], tf.int64)
            })

    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.multiply(tf.subtract(image, 0.5), 2)
    image = tf.reshape(image, [64, 64, 3])
    image = tf.random_crop(image, [54, 54, 3])
    
    length = tf.cast(features['length'], tf.int32)
    digits = tf.cast(features['digits'], tf.int32)
    
    
    
    min_queue_examples = int(0.4 * num_examples)

    image_batch, length_batch, digits_batch = tf.train.batch([image, length, digits],
                                                                     batch_size=batch_size,
                                                                     num_threads=2,
                                                                     capacity=min_queue_examples + 3 * batch_size)
    return image_batch, length_batch, digits_batch

In [3]:
def write_summary(path_to_eval_log_dir):
    summary_writer = tf.summary.FileWriter(path_to_eval_log_dir)
    return summary_writer

def evaluate(writer, path_to_checkpoint, path_to_tfrecords_file, num_examples, global_step):
    batch_size = 128
    num_batches = num_examples // batch_size
    needs_include_length = False

    with tf.Graph().as_default():
        image_batch, length_batch, digits_batch = build_batch(path_to_tfrecords_file,
                                                                         num_examples=num_examples,
                                                                         batch_size=batch_size,
                                                                         shuffled=False)
        length_logits, digits_logits = Model.inference(image_batch, drop_rate=0.0)
        length_predictions = tf.argmax(length_logits, axis=1)
        digits_predictions = tf.argmax(digits_logits, axis=2)


        labels = digits_batch
        predictions = digits_predictions

        labels_string = tf.reduce_join(tf.as_string(labels), axis=1)
        predictions_string = tf.reduce_join(tf.as_string(predictions), axis=1)

        accuracy, update_accuracy = tf.metrics.accuracy(
        labels=labels_string,
        predictions=predictions_string
            )

        tf.summary.image('image', image_batch)
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.histogram('variables',
                                 tf.concat([tf.reshape(var, [-1]) for var in tf.trainable_variables()], axis=0))
        summary = tf.summary.merge_all()

        with tf.Session() as sess:
            sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)

            restorer = tf.train.Saver()
            restorer.restore(sess, path_to_checkpoint)

            for _ in range(num_batches):
                sess.run(update_accuracy)

            accuracy_val, summary_val = sess.run([accuracy, summary])
            writer.add_summary(summary_val, global_step=global_step)

            coord.request_stop()
            coord.join(threads)

    return accuracy_val




In [4]:
#Build the training process

def train(path_to_train_tfrecords_file, num_train_examples, path_to_val_tfrecords_file, num_val_examples,
           path_to_train_log_dir, training_options):
    batch_size = training_options['batch_size']
    initial_patience = training_options['patience']
    num_steps_to_show_loss = 10
    num_steps_to_check = 100

    with tf.Graph().as_default():
        image_batch, length_batch, digits_batch = build_batch(path_to_train_tfrecords_file,
                                                                     num_examples=num_train_examples,
                                                                     batch_size=batch_size,
                                                                     shuffled=True)
        
        length_logtis, digits_logits = Model.inference(image_batch, drop_rate=0.2)
        loss = Model.loss(length_logtis, digits_logits, length_batch, digits_batch)
        
        global_step = tf.Variable(0, name='global_step', trainable=False)
        learning_rate = tf.train.exponential_decay(training_options['learning_rate'], global_step=global_step,
                                                   decay_steps=training_options['decay_steps'], decay_rate=training_options['decay_rate'], staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train_op = optimizer.minimize(loss, global_step=global_step)

        tf.summary.image('image', image_batch)
        tf.summary.scalar('loss', loss)
        tf.summary.scalar('learning_rate', learning_rate)
        summary = tf.summary.merge_all()

        with tf.Session() as sess:
            summary_writer = tf.summary.FileWriter(path_to_train_log_dir, sess.graph)
            evaluator = write_summary(os.path.join(path_to_train_log_dir, 'eval/val'))
            #evaluator = Evaluator(os.path.join(path_to_train_log_dir, 'eval/val'))
    
            #tf.global_variables_initializer()
            sess.run(tf.global_variables_initializer())
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)

            saver = tf.train.Saver()

            print ('Start training')
            patience = initial_patience
            best_accuracy = 0.0
            duration = 0.0
            epoch = 32000
            k=0
            for i in range(epoch):
                k+=1
                start_time = time.time()
                _, loss_val, summary_val, global_step_val, learning_rate_val = sess.run([train_op, loss, summary, global_step, learning_rate])
                duration += time.time() - start_time

                if global_step_val % num_steps_to_show_loss == 0:
                    examples_per_sec = batch_size * num_steps_to_show_loss / duration
                    duration = 0.0
                    print ('%s: step %d, loss = %f ' % (
                        datetime.now(), global_step_val, loss_val))

                if global_step_val % num_steps_to_check != 0:
                    continue

                summary_writer.add_summary(summary_val, global_step=global_step_val)


                path_to_latest_checkpoint_file = saver.save(sess, os.path.join(path_to_train_log_dir, 'latest.ckpt'))
                accuracy = evaluate(evaluator, path_to_latest_checkpoint_file, path_to_val_tfrecords_file,
                                              num_val_examples,
                                              global_step_val)
                print ('Validation accuracy is= %f, best accuracy %f' % (accuracy, best_accuracy))

                if accuracy > best_accuracy:
                    path_to_checkpoint_file = saver.save(sess, os.path.join(path_to_train_log_dir, 'model.ckpt'),
                                                         global_step=global_step_val)
                    print ('Save file to: %s' % path_to_checkpoint_file)
                    patience = initial_patience
                    best_accuracy = accuracy
                else:
                    patience -= 1


                if k>=epoch:
                    break

            coord.request_stop()
            coord.join(threads)

            print ('Training progess is finished')

In [5]:
#Train the model 
#define the folder path
train_tfrecords_file = 'data/train.tfrecords'
val_tfrecords_file = 'data/val.tfrecords'
tfrecords_meta_file = 'data/meta.json'
log_dir = 'logs5/train'
opt = {
    'batch_size': 32,
    'learning_rate': 1e-2,
    'patience': 100,
    'decay_steps': 10000,
    'decay_rate': 0.9
    }
with open(tfrecords_meta_file, 'r') as f:
    content = json.load(f)
    num_train_examples = content['num_examples']['train']
    num_val_examples = content['num_examples']['val']
    num_test_examples = content['num_examples']['test']
    
    
#train the model     
train(train_tfrecords_file, num_train_examples,
           val_tfrecords_file, num_val_examples,
           log_dir, 
           opt)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecor

2019-12-06 16:14:41.163990: step 450, loss = 6.307104 
2019-12-06 16:14:41.962348: step 460, loss = 7.555735 
2019-12-06 16:14:42.762343: step 470, loss = 7.478161 
2019-12-06 16:14:43.542537: step 480, loss = 6.828054 
2019-12-06 16:14:44.355928: step 490, loss = 5.943374 
2019-12-06 16:14:45.141430: step 500, loss = 6.399864 
INFO:tensorflow:Restoring parameters from logs5/train/latest.ckpt
Validation accuracy is= 0.021334, best accuracy 0.020733
Save file to: logs5/train/model.ckpt-500
2019-12-06 16:14:54.359897: step 510, loss = 6.651124 
2019-12-06 16:14:55.154351: step 520, loss = 7.364427 
2019-12-06 16:14:55.943634: step 530, loss = 6.888393 
2019-12-06 16:14:56.738379: step 540, loss = 7.181553 
2019-12-06 16:14:57.529415: step 550, loss = 6.578374 
2019-12-06 16:14:58.345881: step 560, loss = 6.763027 
2019-12-06 16:14:59.127479: step 570, loss = 6.485048 
2019-12-06 16:14:59.929805: step 580, loss = 6.581409 
2019-12-06 16:15:00.710492: step 590, loss = 6.256154 
2019-12-06 

2019-12-06 16:17:43.328136: step 1570, loss = 5.956485 
2019-12-06 16:17:44.144140: step 1580, loss = 4.951407 
2019-12-06 16:17:44.929179: step 1590, loss = 6.115868 
2019-12-06 16:17:45.722839: step 1600, loss = 5.920360 
INFO:tensorflow:Restoring parameters from logs5/train/latest.ckpt
Validation accuracy is= 0.061899, best accuracy 0.051382
Save file to: logs5/train/model.ckpt-1600
2019-12-06 16:17:55.462596: step 1610, loss = 5.659116 
2019-12-06 16:17:56.256027: step 1620, loss = 5.834781 
2019-12-06 16:17:57.079820: step 1630, loss = 5.406193 
2019-12-06 16:17:57.896586: step 1640, loss = 5.594990 
2019-12-06 16:17:58.700324: step 1650, loss = 5.438368 
2019-12-06 16:17:59.499340: step 1660, loss = 5.703244 
2019-12-06 16:18:00.303093: step 1670, loss = 4.950325 
2019-12-06 16:18:01.137714: step 1680, loss = 5.337277 
2019-12-06 16:18:01.931929: step 1690, loss = 4.257630 
2019-12-06 16:18:02.743031: step 1700, loss = 5.614092 
INFO:tensorflow:Restoring parameters from logs5/tra

KeyboardInterrupt: 